In [1]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

In [2]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('groceries_mba.csv', 'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    basket= [x for x in reader]

In [3]:
basket[0], len(basket)

(['citrus_fruit', 'semi-finished_bread', 'margarine', 'ready_soups'], 9835)

In [4]:
basket[:10]

[['citrus_fruit', 'semi-finished_bread', 'margarine', 'ready_soups'],
 ['tropical_fruit', 'yogurt', 'coffee'],
 ['whole_milk'],
 ['pip_fruit', 'yogurt', 'cream_cheese_', 'meat_spreads'],
 ['other_vegetables',
  'whole_milk',
  'condensed_milk',
  'long_life_bakery_product'],
 ['whole_milk', 'butter', 'yogurt', 'rice', 'abrasive_cleaner'],
 ['rolls/buns'],
 ['other_vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled_beer',
  'liquor_(appetizer)'],
 ['pot_plants'],
 ['whole_milk', 'cereals']]

In [5]:
import pickle
with open('data.txt', 'wb') as f:
    pickle.dump(basket, f)

In [6]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [7]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [8]:
wordlist.append('UNKNOWN_TOKEN')

In [9]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [10]:
len(wordlist), len(embeddings)

(170, 170)

In [11]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [12]:
word_to_index

{'Instant_food_products': 99,
 'UHT-milk': 38,
 'UNKNOWN_TOKEN': 169,
 'abrasive_cleaner': 128,
 'artif._sweetener': 131,
 'baby_cosmetics': 163,
 'baby_food': 167,
 'bags': 164,
 'baking_powder': 63,
 'bathroom_cleaner': 139,
 'beef': 26,
 'berries': 40,
 'beverages': 50,
 'bottled_beer': 12,
 'bottled_water': 5,
 'brandy': 124,
 'brown_bread': 18,
 'butter': 24,
 'butter_milk': 48,
 'cake_bar': 74,
 'candles': 93,
 'candy': 44,
 'canned_beer': 14,
 'canned_fish': 72,
 'canned_fruit': 133,
 'canned_vegetables': 81,
 'cat_food': 56,
 'cereals': 111,
 'chewing_gum': 58,
 'chicken': 30,
 'chocolate': 28,
 'chocolate_marshmallow': 91,
 'citrus_fruit': 11,
 'cleaner': 118,
 'cling_film/bags': 78,
 'cocoa_drinks': 144,
 'coffee': 22,
 'condensed_milk': 86,
 'cooking_chocolate': 141,
 'cookware': 138,
 'cream': 153,
 'cream_cheese_': 32,
 'curd': 25,
 'curd_cheese': 119,
 'decalcifier': 152,
 'dental_care': 110,
 'dessert': 36,
 'detergent': 59,
 'dish_cleaner': 84,
 'dishes': 65,
 'dog_food

In [13]:
word_to_index['UNKNOWN_TOKEN']

169

In [14]:
for i, sent in enumerate(basket):
    basket[i] = [w if w in word_to_index else unknown_token for w in sent]

In [15]:
import numpy as np
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in basket])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in basket])

In [16]:
x_example, y_example = X_train[10], y_train[10]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
tropical_fruit other_vegetables white_bread bottled_water
[7, 1, 31, 5]

y:
other_vegetables white_bread bottled_water chocolate
[1, 31, 5, 28]


In [17]:
len(X_train), X_train.shape

(9835, (9835,))

In [18]:
X_train[:10]

array([[11, 64, 21], [7, 4], [], [15, 4, 32], [1, 0, 86], [0, 24, 4, 101],
       [], [1, 38, 2, 12], [], [0]], dtype=object)

In [19]:
y_train[:10]

array([[64, 21, 148], [4, 22], [], [4, 32, 122], [0, 86, 35],
       [24, 4, 101, 128], [], [38, 2, 12, 100], [], [111]], dtype=object)

In [20]:
X_train = np.array([var for var in X_train if var])
y_train = np.array([var for var in y_train if var])

In [21]:
len(X_train), len(y_train)

(7676, 7676)

In [22]:
X_train.shape

(7676,)

In [23]:
basket[10]

['tropical_fruit',
 'other_vegetables',
 'white_bread',
 'bottled_water',
 'chocolate']

In [24]:
np.average([len(row) for row in X_train])

4.3684210526315788

In [25]:
a = [len(row) for row in X_train]
a.index(max(a))

959

In [26]:
import tensorflow as tf
import numpy as np
x = X_train
y = y_train
max_length =  10
x_padded = []
y_padded = []


for row in x:
    if len(row) <= max_length:
        x_padded.append(row + [169] * (max_length - len(row)))
    else :
        x_padded.append(row[:10])

for row in y:
    if len(row) <= max_length:
        y_padded.append(row + [169] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:10])

x_padded = np.array(x_padded)
y_padded = np.array(y_padded)

In [27]:
x_padded.shape, y_padded.shape

((7676, 10), (7676, 10))

In [28]:
y_padded

array([[ 64,  21, 148, ..., 169, 169, 169],
       [  4,  22, 169, ..., 169, 169, 169],
       [  4,  32, 122, ..., 169, 169, 169],
       ..., 
       [ 11,   1,  24, ..., 121,  78, 169],
       [  5,   3,  12, ..., 169, 169, 169],
       [  7,   1, 107, ..., 169, 169, 169]])

In [29]:
len(embeddings)

170

In [30]:
len(x_padded),len(y_padded)

(7676, 7676)

In [31]:
xtrain, ytrain = x_padded[:7000], y_padded[:7000]
xtest, ytest = x_padded[7000:7600], y_padded[7000:7600]

In [32]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 
hm_epochs = 150
batch_size = 500
num_steps = 10
state_size = 200
vocab_size = 170

# Placeholders
x = tf.placeholder(tf.int32, [None, num_steps]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [None])
y = tf.placeholder(tf.int32, [None, num_steps])

def train_neural_network(x): 
    
        def next_batch(step):
            p,q = xtrain[batch_size*step:batch_size*(step+1)], ytrain[batch_size*step:batch_size*(step+1)]
            return p,q
        
        def test_batch(stp):    
            a,b = xtest[batch_size*stp:batch_size*(stp+1)], ytest[batch_size*stp:batch_size*(stp+1)]
            return a,b
              
        def lstm_neural_network(x):
            # Embedding layer
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            rnn_inputs = tf.cast(rnn_inputs, dtype=tf.float32)
            print(rnn_inputs)
            
            # RNN
            inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)   
            cell = tf.contrib.rnn.GRUCell(state_size)
            cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
            print("this is rnn going in:", rnn_inputs)
            rnn_outputs, rnn_states = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)

            rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
            print("this is the output:",rnn_output)
            softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
            softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
            logits = tf.add(tf.matmul(rnn_output, softmax_w),softmax_b, name = "logits")
            return logits

        prediction = lstm_neural_network(x)
        loss = tf.argmax(prediction, axis=1)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [prediction],
            [tf.reshape(y[:batch_size], [-1])],
            [tf.ones([batch_size * num_steps], dtype=tf.float32)])
        cost = tf.reduce_sum(loss) / batch_size
        optimizer = tf.train.AdamOptimizer().minimize(cost)   
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(hm_epochs):
                epoch_loss = 0 
                for step in range(int(len(xtrain)/batch_size)):
                    epoch_x, epoch_y = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./model")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(xtest)/batch_size)):
                s,u = test_batch(stp)
                correct = tf.equal((tf.argmax(prediction,1)),tf.cast(tf.reshape(tf.concat(axis=1, values= u), [-1]),tf.int64))
                accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
                print( 'Batch #', stp, 'Accuracy by each batch:',accuracy.eval({x: s, y: u}))

In [33]:
train_neural_network(x)

Tensor("Cast:0", shape=(?, 10, 200), dtype=float32)
this is rnn going in: Tensor("Cast:0", shape=(?, 10, 200), dtype=float32)
this is the output: Tensor("Reshape:0", shape=(?, 200), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 150 loss: 556.398431778
Epoch 1 completed out of 150 loss: 347.513626099
Epoch 2 completed out of 150 loss: 299.957519531
Epoch 3 completed out of 150 loss: 275.29476738
Epoch 4 completed out of 150 loss: 257.599298477
Epoch 5 completed out of 150 loss: 243.808303833
Epoch 6 completed out of 150 loss: 233.546654701
Epoch 7 completed out of 150 loss: 225.96871376
Epoch 8 completed out of 150 loss: 220.064199448
Epoch 9 completed out of 150 loss: 215.112513542
Epoch 10 completed out of 150 loss: 211.90826416
Epoch 11 completed out of 150 loss: 208.698615074
Epoch 12 completed out of 150 loss: 206.27856636
Epoch 13 completed out of 150 loss: 203.964692116
Epoch 14 completed out of 150 loss: 202.261

In [72]:
tt = np.array(x_padded[7606:7676])
tt[10:20] =  np.array(x_padded[7606:7616])

In [73]:
tt[10:20] = np.array([[  7,   0,  54,  17,  19,   21,  2,   5, 169, 169],
       [  19,  9,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 86,   22,  2, 169, 169, 169, 169, 169, 169, 169],
       [  10,  0,  50, 169, 169, 169, 169, 169, 169, 169],
       [ 2,   0,   26,  81,  16,  33,  28, 169, 169, 169],
       [ 11,  30, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 46,  29, 169, 169, 169, 169, 169, 169, 169, 169],
       [  2,   0, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 39,   1,   18,  4,  10,  56, 128, 169, 169, 169],
       [ 20,   1,   0,   7,  68, 119,  56,  41, 169, 169]])

In [74]:
tt[:30]

array([[  7,   0,  17,  54,  19,   2,  21,   5, 169, 169],
       [  9,  19,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 86,   2,  22, 169, 169, 169, 169, 169, 169, 169],
       [  0,  50,  10, 169, 169, 169, 169, 169, 169, 169],
       [ 26,   0,   2,  81,  16,  33,  28, 169, 169, 169],
       [ 30,  11, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 29,  46, 169, 169, 169, 169, 169, 169, 169, 169],
       [  0,   2, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 39,   1,   4,  18,  10,  56, 128, 169, 169, 169],
       [ 20,   7,   1,   0,  68, 119,  56,  41, 169, 169],
       [  7,   0,  54,  17,  19,  21,   2,   5, 169, 169],
       [ 19,   9,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 86,  22,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 10,   0,  50, 169, 169, 169, 169, 169, 169, 169],
       [  2,   0,  26,  81,  16,  33,  28, 169, 169, 169],
       [ 11,  30, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 46,  29, 169, 169, 169, 169, 169, 169, 169, 169

In [75]:
wordlist[54]

'sliced_cheese'

In [76]:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('model.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

#context = graph.get_tensor_by_name("context:0") 
#utterance = graph.get_tensor_by_name("utterance:0")
#target = graph.get_tensor_by_name("target:0")

op_to_restore = graph.get_tensor_by_name("logits:0")
#qe,an,lbl = test_batch(0)
feed_dict={x: tt}
results = sess.run(op_to_restore,feed_dict)
print("These are results:", results)
results = np.reshape(results, [70, num_steps, vocab_size])

These are results: [[ 0.00754669  0.44657511 -0.12729192 ...,  0.58057088  0.33615345
  -0.02635901]
 [-0.0241589   0.5581727  -0.41380587 ...,  0.43316352  0.33038932
  -0.30631232]
 [-0.25544032  0.08803369 -0.18636683 ..., -0.04221949  0.2560516
  -0.2356609 ]
 ..., 
 [-0.17675501 -0.05529821 -0.01411376 ...,  0.33081028 -0.14848122
   0.17498448]
 [-0.02346693  0.14027222  0.06377    ...,  0.06751322 -0.17026943
   0.23834445]
 [-0.12125824  0.16531292 -0.13084522 ...,  0.21480925 -0.12515812
   0.25546247]]


In [77]:
r1= results.argmax(axis=2)[:20]
r2=tt[:20]

In [78]:
r1

array([[167,  10,  78,  79,  34,  94, 132, 155,   0,  66],
       [ 82,  82, 132, 139,  30, 113, 108, 118, 108, 139],
       [101,  94, 139,  54,  78, 134,  78, 115,  95,  89],
       [132, 108, 157,  43,  43,  43, 108, 108, 108, 123],
       [135,  35,  38,  97, 102, 169, 129, 169, 169, 167],
       [151,   1, 124,  99,  90,  29,  97,  97,  50, 123],
       [ 43,  82,  82,  82,  82,  82,  82,  82,  82,  82],
       [ 90, 132, 130,  31,  94, 108, 108, 147, 108,  29],
       [  5,  34,  34,   6,  99,  56, 160,  37,  44, 158],
       [115,  51,  34,  97,  97,  34,  79,  79,  79,  56],
       [  1, 108,  70,  30,  94,   4,  43, 129,   0, 134],
       [  6,  82, 143,  52,  98, 139,  81, 113,  74, 115],
       [159,  95, 130, 130, 130,  79,  79,  89, 115,  89],
       [117,  56,  21, 154, 108, 123,   5, 108,  21,  43],
       [116,  55,  85,  53,  46, 167, 169,  53, 167, 167],
       [ 26,  29, 124,  29, 108,  50, 108, 167,  14,  50],
       [ 29,  82,  50,  82, 115,  82,  82, 116,  56,  82

In [79]:
r2

array([[  7,   0,  17,  54,  19,   2,  21,   5, 169, 169],
       [  9,  19,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 86,   2,  22, 169, 169, 169, 169, 169, 169, 169],
       [  0,  50,  10, 169, 169, 169, 169, 169, 169, 169],
       [ 26,   0,   2,  81,  16,  33,  28, 169, 169, 169],
       [ 30,  11, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 29,  46, 169, 169, 169, 169, 169, 169, 169, 169],
       [  0,   2, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 39,   1,   4,  18,  10,  56, 128, 169, 169, 169],
       [ 20,   7,   1,   0,  68, 119,  56,  41, 169, 169],
       [  7,   0,  54,  17,  19,  21,   2,   5, 169, 169],
       [ 19,   9,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 86,  22,   2, 169, 169, 169, 169, 169, 169, 169],
       [ 10,   0,  50, 169, 169, 169, 169, 169, 169, 169],
       [  2,   0,  26,  81,  16,  33,  28, 169, 169, 169],
       [ 11,  30, 169, 169, 169, 169, 169, 169, 169, 169],
       [ 46,  29, 169, 169, 169, 169, 169, 169, 169, 169

In [ ]:
root_vegetables,salt,pudding_powder,meat,turkey,bathroom_cleaner,canned_vegetables,sauces,cake_bar,jam

salt,salt,syrup,bathroom_cleaner,chicken,sauces,female_sanitary_products,cleaner,female_sanitary_products,bathroom_cleaner

In [80]:
for pre,the in zip(r1,r2):
    print("Starts")
    print("\ny': These are the predictions made \n%s" % (",".join([wordlist[x] for x in pre])))
    print("\ny These are the ground truths :\n%s" % (",".join([wordlist[x] for x in the])))
    print("End\n")

Starts

y': These are the predictions made 
baby_food,pastry,cling_film/bags,spread_cheese,salty_snack,dog_food,syrup,hair_spray,whole_milk,flour

y These are the ground truths :
tropical_fruit,whole_milk,whipped/sour_cream,sliced_cheese,domestic_eggs,rolls/buns,margarine,bottled_water,UNKNOWN_TOKEN,UNKNOWN_TOKEN
End

Starts

y': These are the predictions made 
salt,salt,syrup,bathroom_cleaner,chicken,sauces,female_sanitary_products,cleaner,female_sanitary_products,bathroom_cleaner

y These are the ground truths :
sausage,domestic_eggs,rolls/buns,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN
End

Starts

y': These are the predictions made 
rice,dog_food,bathroom_cleaner,sliced_cheese,cling_film/bags,nut_snack,cling_film/bags,jam,specialty_cheese,photo/film

y These are the ground truths :
condensed_milk,rolls/buns,coffee,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN
End

Starts

y': 

In [ ]:
white_bread,cocoa_drinks,pastry,nuts/prunes,hard_cheese,waffles,hard_cheese,chicken,margarine,beef

In [42]:
result = np.reshape(results, [-1, vocab_size])

In [43]:
result.shape

(700, 170)

In [44]:
result = sess.run(tf.nn.softmax(result,0))

In [45]:
qa = np.reshape(y_padded[7606:7676], [-1])

In [46]:
qa.shape

(700,)

In [47]:
qb = result.argmax(axis=1)

In [48]:
qb.shape

(700,)

In [49]:
b=0
for i,j in enumerate(qa):
    b += -1*np.log(result[i][j])
    print(i,j,b,result[i][j])

0 0 6.25392818451 0.00192289
1 17 12.6874060631 0.00160685
2 54 18.8751335144 0.00205449
3 19 25.4457855225 0.00140088
4 2 32.3433198929 0.00101027
5 21 38.1614384651 0.00297319
6 5 44.2671031952 0.0022302
7 16 50.6720347404 0.00165338
8 169 57.5053930283 0.00107723
9 169 64.0146169662 0.00148963
10 19 71.0393280983 0.000889625
11 2 77.6624069214 0.00132933
12 18 84.3062157631 0.00130206
13 169 90.9073314667 0.00135885
14 169 97.4097781181 0.00149977
15 169 103.993902206 0.00138214
16 169 110.77888298 0.00113063
17 169 117.52573204 0.00117457
18 169 124.119570255 0.00136878
19 169 130.760578632 0.00130571
20 2 137.57756424 0.00109502
21 22 144.312800407 0.00118829
22 13 151.310464382 0.000914014
23 169 157.925091267 0.00134061
24 169 164.591121674 0.00127344
25 169 171.298449039 0.00122193
26 169 178.045204163 0.00117468
27 169 184.642576694 0.00136395
28 169 191.292489052 0.00129414
29 169 197.93434 0.00130461
30 50 204.386339188 0.00157737
31 10 211.397754192 0.000901532
32 13 217.38

In [50]:
980/150

6.533333333333333